# Dashboard Interativo com Panel
Este notebook apresenta um dashboard interativo baseado no dataset TMDB Movie Dataset.
Inclui filtros, visualizações e estatísticas descritivas.


In [ ]:
import pandas as pd
import panel as pn
import hvplot.pandas
pn.extension('tabulator')

# Carregar dataset
file_path = 'TMDB_movie_dataset_v11 - Copy.csv'
df = pd.read_csv(file_path)

# Pré-visualização
df.head()

In [ ]:
# Limpeza de dados
# Converter release_date para datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Remover duplicados
df.drop_duplicates(inplace=True)

# Preencher valores nulos em revenue e budget com 0
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)

# Criar coluna de ano
df['release_year'] = df['release_date'].dt.year


In [ ]:
# Estatísticas descritivas
df.describe(include='all')

In [ ]:
# Dashboard interativo
# Widgets
genre_filter = pn.widgets.Select(name='Gênero', options=['Todos'] + sorted(set(', '.join(df['genres'].dropna()).split(', '))))
year_slider = pn.widgets.IntSlider(name='Ano de Lançamento', start=int(df['release_year'].min()), end=int(df['release_year'].max()), step=1, value=int(df['release_year'].min()))
language_filter = pn.widgets.Select(name='Idioma', options=['Todos'] + sorted(df['original_language'].dropna().unique().tolist()))

# Função de filtro
def filter_data(genre, year, language):
    data = df.copy()
    if genre != 'Todos':
        data = data[data['genres'].str.contains(genre, na=False)]
    if language != 'Todos':
        data = data[data['original_language'] == language]
    if year:
        data = data[data['release_year'] == year]
    return data

# Tabela dinâmica
table = pn.bind(lambda g, y, l: filter_data(g, y, l).hvplot.table(), genre_filter, year_slider, language_filter)

# Gráficos
plot_votes = pn.bind(lambda g, y, l: filter_data(g, y, l).hvplot.hist('vote_average', bins=20, title='Distribuição das Avaliações'), genre_filter, year_slider, language_filter)
plot_revenue = pn.bind(lambda g, y, l: filter_data(g, y, l).hvplot.scatter(x='budget', y='revenue', title='Budget vs Revenue'), genre_filter, year_slider, language_filter)

# Layout do dashboard
dashboard = pn.Column(
    pn.Row(genre_filter, year_slider, language_filter),
    pn.Tabs(('Tabela', table), ('Distribuição de Avaliações', plot_votes), ('Budget vs Revenue', plot_revenue))
)

dashboard.servable()